In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import yfinance as yf

In [ ]:
stock_list = {
    'VUKE.L': 'FTSE 100',
    'VMID.L': 'FTSE 250',
    'VUSA.L': 'S&P 500',
    'VERX.L': 'DEVELOPED EUROPE EX UK',
    'VEUR.L': 'DEVELOPED EUROPE',
    'VGER.L': 'GERMANY ALL CAP',
    #'V3AM.L': 'ESG GLOBAL ALL CAP',
    'VWRL.L': 'FTSE ALL WORLD',
    'VHYL.L': 'WORLD HIGH DIVIDEND YIELD',
    'VEVE.L': 'DEVELOPED WORLD',
    'VJPN.L': 'FTSE JAPAN',
    #'VAPX.L': 'ASIA EX JAPAN',
    'VNRT.L': 'FTSE NORTH AMERICA',
    'VFEM.L': 'EMERGING MARKETS'
}
#stock_list = ['VUKE.L', 'VMID.L', 'VUSA.L', 'VERX.L', 'VEUR.L', 'VGER.L', 'V3AM.L', 'VWRL.L', 'VEVE.L', 'VJPN.L','VAPX.L', 'VNRT.L', 'VFEM.L']
fixed_income = ['VECP.L', 'VETY.L', 'VAGP.L', 'VGOV.L', 'VUCP.L', 'VUTY.L', 'VEMT.L']

stocks = yf.Tickers(list(stock_list.keys()))
stock_prices = stocks.history('5y')['Close']
#stock_prices.dropna(inplace=True)
#vuke = vuke.history('5y')

In [ ]:
stock_prices

In [ ]:
n_days = 4000
n_stocks = 2000

In [ ]:
# Simulate daily returns for all stocks over the course of our observation period.


mu = 0.001
sigma = 0.1

np.random.seed(0)
returns = returns = np.random.normal(mu, sigma, (n_stocks, n_days))
print(f'returns.shape={returns.shape}')

market_caps = np.random.uniform(10000,1000000, n_stocks)


In [ ]:
# Calculate the market portfolio return series.

cumulative_return = np.array([np.cumsum(1+returns[i]) for i in range(n_stocks)])

# market portfolio returns
market_cap_series = np.array([cumulative_return[i]*market_caps[i] for i in range(n_stocks)])

print(f'market_cap_series.shape={market_cap_series.shape}')

weights = np.array([market_cap_series[i]/sum(market_cap_series[i]) for i in range(n_stocks)])

print(f'weights.shape={weights.shape}')

weights_returns = weights*returns

print(f'weights_returns.shape={weights_returns.shape}')

market_return = np.array([sum(weights_returns[j][i] for j in range(n_stocks)) for i in range(n_days)])

print(f'market_return.shape={market_return.shape}')


In [ ]:
# conduct CAPM analysis for each stock and record the beta.

from sklearn.linear_model import LinearRegression

betas = []

for i in range(n_stocks):
    regressinoModel = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=3)
    y = returns[:][i]
    x = market_return.reshape(-1,1)
    regressinoModel.fit(x,y)
    # intercept, beta, index 
    betas.append((regressinoModel.intercept_,regressinoModel.coef_[0], i))



In [ ]:
# Sort the betas
betas.sort(key=lambda it: it[1])


# split into low and high sets of beta
lowBeta = betas[0:int(n_stocks/2)-1]
highBeta = betas[int(n_stocks/2):n_stocks-1]

# subset the returns matrix into high and low beta stocks.
lowBetaReturns = np.array([returns[j[2]] for j in lowBeta])
highBetaReturns = np.array([returns[j[2]] for j in highBeta])
print(lowBetaReturns.shape)

# equally weight each asset and compute the portfolio returns
lowReturns = np.array([sum(1/1000*lowBetaReturns[...,i]) for i in range(n_days)])
highReturns = np.array([sum(1/1000*highBetaReturns[...,i]) for i in range(n_days)])


In [ ]:
# Calculate the H and L portfolio betas

lowRetModel = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=3)
lowRetModel.fit(y=lowReturns, X=market_return.reshape(-1,1))

highRetModel = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=3)
highRetModel.fit(y=highReturns, X=market_return.reshape(-1,1))


# show that they are equal to the average beta in the simple case.
print(f'=== Simple average ===')
print(f'Average low beta {sum([stat[1] for stat in lowBeta])/1000}')
print(f'Average high beta {sum([stat[1] for stat in highBeta])/1000}')
print(f'=== CAPM ===')
print(f'LowBeta: {lowRetModel.coef_}, Alpha: {lowRetModel.intercept_}')
print(f'HighBeta: {highRetModel.coef_}, Alpha: {highRetModel.intercept_}')

lowBetaVal = lowRetModel.coef_[0]
highBetaVal = highRetModel.coef_[0]

scalarLow = 1/lowBetaVal
scalarHigh = 1/highBetaVal
print('')

print(f'For every dollar invested in lowBeta, we will leverage by {scalarLow}')
print(f'For every dollar in highBeta, we will delveraged by, {scalarHigh}')

bab_portfolio = (scalarLow * lowReturns) - (scalarHigh * highReturns)


In [ ]:
plt.plot(np.cumprod(1+bab_portfolio), label='BAB Portfolio')
plt.plot(np.cumprod(1+market_return), label='Market Portfolio')
plt.legend()

In [ ]:
regressionModel = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=3)
regressionModel.fit(y=bab_portfolio, X=market_return.reshape(-1,1))

print(f'Beta coefficient: {regressionModel.coef_[0]} ~= 0')
print(f'Intercept (Alpha): {regressionModel.intercept_}')